# **Measure Object Morphology**

***Prior to this notebook, you should have already run through [2.0_quantification_setup](2.0_quantification_setup.ipynb).***

The `'methods_...'` notebooks included here in infer-subc Part 2: quantification will go over how each of the quantification methods (morphology, interactions, and distribution) are carried out. The notebooks will explain each step in the method and display the combined function at the end of the notebook. 

### **Biological Relevance**
Measurements of organelle, interaction site, and cell/cellular region morphology -- amount, size, and shape -- are included as part of the organelle signature analysis. These metrics can provide information about the physiology of a cell and its constituent organelles. 

Organelle amount per cell has been demonstrated to differ between cell types. The cytoplasms of some specialized cell types, like adipocytes, are composed almost entirely by a single large lipid droplet for fat storage[[1]](https://doi.org/10.3390/biom11121906), while other cell types, like muscle cells, have large and elaborate mitochdondrial networks for effective metabolite diffusion during muscle contraction[[2]](https://doi.org/10.1038/nature14614).

Additionally, different organelle morphologies are important to their function. For example, fission and fusion maintain mitochondrial homeostasis by modulating the size and connectedness of the mitochondrial network. A recent study demonstrated that asymmetric fission events resulted in morphologically distinct daughter mitochondria with different fates; the larger mitochondria continued to grow and divide, while the smaller, spherical fragments containing high reactive oxygen species (ROS) were destined for autophagic degradation[[3]](https://doi.org/10.1038/s41586-021-03510-6).

The size and shape of cells can also be informative about their functions. For example, the shape of astrocyte cells and nuclei can indicate their activity state (e.g., reactive or non-reactive) and function[[4]](https://doi.org/10.1186/s40659-024-00532-y). 

### **Morphology Measurements** 📐
The following morphological measurements are included:
- `label`: the unique ID number for the object being measured
- `centroid`: centroid coordinate tuple (row, col, Z)
- `bbox`: bounding box coordinates (min_row, min_col, max_row, max_col); pixels/voxels belonging to the bounding box are in the half-open interval [min_row; max_row) and [min_col; max_col).
- `area`: (or `volume` for 3D z-stack images) area of the region i.e. number of pixels of the region scaled by pixel-area; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `equivalent_diameter`: the diameter of a circle with the same area as the region; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `extent`: ratio of pixels/voxels in the region to pixels/voxels in the total bounding box. Computed as area / (rows * cols)
- `euler_number`: Euler characteristic of the set of non-zero pixels. Computed as number of connected components subtracted by number of holes (input.ndim connectivity). In 3D, number of connected components plus number of holes subtracted by number of tunnels.
- `solidity`: ratio of pixels/voxels in the region to pixels/voxels of the convex hull image.
- `axis_major_length`: the length of the major axis of the ellipse that has the same normalized second central moments as the region; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `surface_area`: the surface area of the region. For 3D, surface area of a 2D surface mesh of the region (skimage.measure.marching_cubes) using skimage.measure.mesh_surface_area; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `SA_to_volume`: surface area / area (or volume); this metric has the option to be converted into "real world" units using the scale from the metadata.

The following measures of the intensity images are also included:
- `min_intensity`: value with the least intensity in the region.
- `max_intensity`: value with the greatest intensity in the region.
- `mean_intensity`: value with the mean intensity in the region.
- `standard_deviation_intensity`: the standard deviation of the intensity in the region.

These measurements and definitions are derived from the [`skimage.measure.regionprops()`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops) function. More in depth information about each measurement can be found there.

-----

### 👣 **Summary of steps**  

🛠️ **BUILD FUNCTION STEP-BY-STEP**

- **`1`** - Apply mask to the object of interest

- **`2`** - Build the list of measurements we want to include from regionprops 

- **`3`** - Add additional measurements as *"extra_properties"* with custom functions.

- **`4`** - Run regionprops and export values as a pandas dataframe

- **`5`** - Add additional measurements not compatible with regionprops

- **`6` - Rename and add additional metadata columns**

⚙️ **DEFINE AND TEST `*Morphology*` FUNCTION**

- Define `get_morphology_metrics` function
- Run `get_morphology_metrics` function

The above steps will be applied to a single organelle of interest. Batch processing is available in a separate notebook.

---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`2.0_quantification_setup`](2.0_quantification_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

In [18]:
from pathlib import Path
import os

import napari
from napari.utils.notebook_display import nbscreenshot

from skimage.measure import (regionprops, regionprops_table)

from infer_subc.core.file_io import (read_czi_image,
                                     import_inferred_organelle,
                                     list_image_files)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats import (_assert_uint16_labels)
from infer_subc.utils.stats_helpers import *
from infer_subc.organelles import * 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### &#x1F3C3; **Run code; no user input required**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `raw_img_type`, `data_root_path`, `raw_data_path`, `seg_data_path`, and `quant_data_path`.

In [19]:
#### USER INPUT REQUIRED ###
raw_img_type = ".czi"
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"
raw_data_path = data_root_path / "raw_single"
seg_data_path = data_root_path / "out_single"
quant_data_path = data_root_path / "quant_single"

#### &#x1F3C3; **Run code; no user input required**

In [20]:
# Create the output directory to save the segmentation outputs in.
if not Path.exists(quant_data_path):
    Path.mkdir(quant_data_path)
    print(f"making {quant_data_path}")

# Create a list of the file paths for each image in the input folder. Select test image path.
raw_img_file_list = list_image_files(raw_data_path,raw_img_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":raw_img_file_list})

,Image Name
0,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_single\a24hrs_Ctrl_14_Unmixing.czi


#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [21]:
#### USER INPUT REQUIRED ###
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [22]:
# Read in the image and metadata as an ndarray and dictionary from the test image selected above. 
test_img_name = raw_img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# Define some of the metadata features.
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']

print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

Metadata information
File path: C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_single\a24hrs_Ctrl_14_Unmixing.czi
Channel 0 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Nuclei_Jan22
Channel 1 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Lyso+405_Jan22
Channel 2 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Mito+405_Jan22
Channel 3 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Golgi+405_Jan22
Channel 4 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Peroxy+405_Jan22
Channel 5 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: ER+405_Jan22
Channel 6 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: BODIPY+405low_Jan22
Channel 7 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Residuals
Scale (ZYX): (0.3891184878080979, 0.07987165184837317, 0.07987165184837318)
Channel axis: 0


#### &#x1F6D1; &#x270D; **User Input Required:**

Specify the following information about the segmentation files: - `org_file_names`, `org_channels_ordered`, `regions_file_names`, and `suffix_separator`.

In [23]:
#### USER INPUT REQUIRED ###
org_file_names = ["lyso", "mito", "golgi", "perox", "ER", "LD"]
org_channels_ordered = [1, 2, 3, 4, 5, 6]
regions_file_names = ["cell", "nuc"]
suffix_separator = "-"

#### &#x1F3C3; **Run code; no user input required**

In [24]:
# find file paths for segmentations
all_suffixes = org_file_names + regions_file_names
filez = find_segmentation_tiff_files(file_path, all_suffixes, seg_data_path, suffix_separator)

# read the segmentation and masks/regions files into memory
organelles = [read_tiff_image(filez[org]) for org in org_file_names]
regions = [] 
for m in regions_file_names:
    mfile = read_tiff_image(filez[m])
    regions.append(mfile)

# match the intensity channels to the segmentation files
intensities = [img_data[ch] for ch in org_channels_ordered]

# open viewer and add images
viewer = napari.Viewer()
for r, reg in enumerate(regions_file_names):
    viewer.add_image(regions[r],
                     scale=scale,
                     name=f"{reg} mask")

# colors = ["red", "bop orange", "yellow", "green", "blue", "cyan", "magenta", "bop purple"]
for o, org in enumerate(org_file_names):
    viewer.add_image(intensities[o],
                     scale=scale,
                     name=f"{org} intensity channel")
    viewer.add_labels(organelles[o],
                      scale=scale,
                      name=f"{org} segmentation")
viewer.grid.enabled = True
viewer.reset_view()

print("The following matching files were found and can now be viewed in Napari:")
filez

03-Dec-24 11:22:04 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1086x661+1532+199 (frame: 1102x700+1524+168) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY2". Resulting geometry: 867x527+1531+192 (frame: 883x566+1523+161) margins: 8, 31, 8, 8 minimum size: 385x513 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=401,552 maxtrack=0,0)


The following matching files were found and can now be viewed in Napari:


{'raw': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw_single/a24hrs_Ctrl_14_Unmixing.czi'),
 'lyso': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-lyso.tiff'),
 'mito': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-mito.tiff'),
 'golgi': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-golgi.tiff'),
 'perox': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-perox.tiff'),
 'ER': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-ER.tiff'),
 'LD': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-LD.tiff'),
 'cell': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-cell.tiff'),
 'nuc': WindowsPath('C:/Use

-----

## **APPLY MASK**

### **`STEP 1` - Apply mask to the object of interest**

&#x1F453; **FYI:** To ensure we are performing single cell analysis, we will apply the cell segmentation as a mask to the segmentation file. This will exclude any objects outside of the mask area from the analysis.

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify which organelle you would like to examine in this analysis and which segmentation file should be used as the mask:
- `obj_seg`: the suffix of the object you would like to measure morphological features of; it should match one of the names included in the `org_file_names` variable above
- `mask`: the suffix of the mask you would like to use; it should match one of the names included in the `regions_file_names` variable above. This area will define the area from whic objects will be measured; anyhting outside of the mask will be ignored. 

In [25]:
#### USER INPUT REQUIRED ###
obj_seg_name = "golgi"
mask_name = "cell"

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** The files are read into memory and the mask is applied. The objects before (magenta) and after (green) masking can be viewed in Napari. The objects included in the mask will appear white (a combination of the magenta and green colors) and the mask area will appear grey.

In [26]:
# read segmentation and intensity files into memory
o = org_file_names.index(obj_seg_name)
seg = organelles[o]
int = intensities[o]

m = regions_file_names.index(mask_name)
mask = regions[m]

# apply mask to segmentation image
masked_seg = apply_mask(seg, mask)

# open in Napari
viewer.layers.clear()
viewer.grid.enabled = False
viewer.add_image(mask, scale=scale, name="Mask", opacity=0.7)
viewer.add_image(int, scale=scale, name="Raw Intensity Channel", blending="additive")
viewer.add_image(seg>0, scale=scale, name="Original Segmentation Output", blending="translucent", opacity=0.7, colormap="magenta")
viewer.add_image(masked_seg>0, scale=scale, name="Masked Segmentation", blending="additive", colormap="green")
viewer.reset_view

<bound method ViewerModel.reset_view of Viewer(camera=Camera(center=(0.0, 28.07488562470317, 28.074885624703175), zoom=15.678563023211515, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(3.112947902464783, 1.0, 0.0), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=3, ndisplay=2, order=(0, 1, 2), axis_labels=('0', '1', '2'), rollable=(True, True, True), range=(RangeTuple(start=0.0, stop=6.225895804929566, step=0.3891184878080979), RangeTuple(start=0.0, stop=56.149771249406335, step=0.07987165184837317), RangeTuple(start=0.0, stop=56.14977124940634, step=0.07987165184837318)), margin_left=(0.0, 0.0, 0.0), margin_right=(0.0, 0.0, 0.0), point=(3.112947902464783, 28.03494979877898, 28.034949798778985), last_used=0), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Mask' at 0x18808e9c6a0>, <Image layer 'Raw Intensity Channel' at 0x18807f87310>, <Image layer 'Original Segmentati

### **`Step 2` - Build the list of measurements we want to include from regionprops**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This block of code lists all of the [`skimage.measure.regionprops`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops) properties that we will include in the analysis. The final list of properties are printed below.

In [27]:
# start with LABEL
test_properties = ["label"]

# add position
test_properties = test_properties + ["centroid", "bbox"]

# add area
test_properties = test_properties + ["area", "equivalent_diameter"]

# add shape measurements
test_properties = test_properties + ["extent", "euler_number", "solidity", "axis_major_length"]

# add intensity values
test_properties = test_properties + ["min_intensity", "max_intensity", "mean_intensity"]

print("The following properties will be included:")
test_properties

The following properties will be included:


['label',
 'centroid',
 'bbox',
 'area',
 'equivalent_diameter',
 'extent',
 'euler_number',
 'solidity',
 'axis_major_length',
 'min_intensity',
 'max_intensity',
 'mean_intensity']

### **`Step 3 ` - Add additional measurements as *"extra_properties"* with custom functions**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This block of code creates additional properties that can be included through regionprops.

In [28]:
# create a function to include the standard deviation of intensities
def standard_deviation_intensity(region, intensities):
    return np.std(intensities[region])

test_extra_properties = [standard_deviation_intensity]

### **`Step 4` - Run regionprops and export values as a pandas dataframe**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block runs the regionprops function including the build in measurements and additional metrics included in extra_properties.

The function is run with the scale to produce "real world" units and without the scale (pixel/voxel units) for comparison.

In [41]:
### SCALED ###
# run regionprops with the scale
test_props = regionprops_table(masked_seg,
                               intensity_image=int, 
                               properties=test_properties,
                               extra_properties=test_extra_properties,
                               spacing=scale)

### UNSCALED ###
# run regionprops without the scale
test_props_unscaled = regionprops_table(masked_seg,
                            intensity_image=int, 
                            properties=test_properties,
                            extra_properties=test_extra_properties)

### **`Step 5` - Add additional measurements not compatable with regionprops**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block creates a function to measure the surface area of an object from the regionprops output. It

The function is run with the scale to produce "real world" units and without the scale (pixel/voxel units) for comparison.

In [42]:
# creating a function to measure the surface area of each object. This function utilizes "marching_cubes" to generate a mesh (non-pixelated object)
def _surface_area_from_props(labels: np.ndarray,
                             props: dict,
                             scale: Union[tuple, None]=None):
    """ 
    a function for getting surface area of volumetric objects

    Parameters:
    ----------
    lables:
        the segmentation np.ndarray with each object labeled a different number
    props:
        region props dictionary resulting from the _my_props_to_dict() function
    spacing:
        tuple of the dimension lengths in the same order as the dimension of your np.ndarray labels input
    """
    surface_areas = np.zeros(len(props["label"]))

    for index, lab in enumerate(props["label"]):
        # this seems less elegant than you might wish, given that regionprops returns a slice,
        # but we need to expand the slice out by one voxel in each direction, or surface area freaks out
        volume = labels[
            max(props["bbox-0"][index] - 1, 0) : min(props["bbox-3"][index] + 1, labels.shape[0]),
            max(props["bbox-1"][index] - 1, 0) : min(props["bbox-4"][index] + 1, labels.shape[1]),
            max(props["bbox-2"][index] - 1, 0) : min(props["bbox-5"][index] + 1, labels.shape[2]),
        ]
        volume = volume == lab
        if scale is None:
            scale=(1.0,) * labels.ndim
        verts, faces, _normals, _values = marching_cubes(volume,
                                                         method="lewiner",
                                                         spacing=scale,
                                                         level=0)
        
        surface_areas[index] = mesh_surface_area(verts, faces)

    return surface_areas

# run surface area function scaled, then unscaled
test_SA_tab = pd.DataFrame(_surface_area_from_props(masked_seg, test_props, scale))

test_SA_tab_unscaled = pd.DataFrame(_surface_area_from_props(masked_seg, test_props))

### **`Step 6` - Rename and add additional metadata columns**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block adds additional metadata information to the table and updates some column names for clarity. The scaled and unscaled tables are printed below for comparison.

In [46]:
### SCALED ###
# edited column names and add the scale
test_props_table = pd.DataFrame(test_props)
test_props_table.insert(0, "object", obj_seg_name)
test_props_table.rename(columns={"area": "volume"}, inplace=True)
test_props_table.insert(loc=2, column="scale", value=f"{(round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))}")
test_props_table.insert(12, "surface_area", test_SA_tab)
test_props_table.insert(14, "SA_to_volume_ratio", test_props_table["surface_area"].div(test_props_table["volume"]))

### UNSCALED ###
# edited column names and add the scale
test_props_table_unscaled = pd.DataFrame(test_props_unscaled)
test_props_table_unscaled.insert(0, "object", obj_seg_name)
test_props_table_unscaled.rename(columns={"area": "volume"}, inplace=True)
test_props_table_unscaled.insert(loc=2, column="scale", value=f"{tuple(np.ones(masked_seg.ndim))}")
test_props_table_unscaled.insert(12, "surface_area", test_SA_tab_unscaled)
test_props_table_unscaled.insert(14, "SA_to_volume_ratio", test_props_table_unscaled["surface_area"].div(test_props_table_unscaled["volume"]))

# set pandas dataframe to display all columns
pd.set_option('display.max_columns', None)
print("Morphology measurement (scaled):")
display(test_props_table)
print("Morphology measurement (unscaled):")
display(test_props_table_unscaled)

Morphology measurement (scaled):


,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,golgi,1,"(0.3891, 0.0799, 0.0799)",3.047327,25.376828,18.171967,1,274,181,16,371,280,153.181577,33.752838,4.538332,4.009626,0.094394,1,0.274931,10.776564,0.0,65535.0,18106.068030,11022.022519
1,golgi,3,"(0.3891, 0.0799, 0.0799)",3.561908,19.649771,18.060408,2,221,207,16,272,255,59.918405,13.268289,4.515911,2.937235,0.155958,1,0.460220,5.637385,0.0,65535.0,24548.568756,14668.560908
2,golgi,4,"(0.3891, 0.0799, 0.0799)",1.387752,25.729279,14.372799,2,305,171,6,342,192,15.822502,2.467480,6.412414,1.676557,0.319820,1,0.822167,3.121573,0.0,39152.0,10127.904427,7046.602988
3,golgi,5,"(0.3891, 0.0799, 0.0799)",1.273343,26.119869,19.036417,2,319,230,6,336,248,10.881411,1.941216,5.605460,1.547718,0.638889,1,0.899885,1.793660,0.0,37253.0,14053.263427,8411.557879
4,golgi,6,"(0.3891, 0.0799, 0.0799)",1.962179,22.591572,16.149313,3,266,190,9,300,216,20.326122,3.668949,5.540040,1.913585,0.278658,1,0.773822,3.186420,0.0,50751.0,13597.763870,10113.348468
5,golgi,7,"(0.3891, 0.0799, 0.0799)",1.949098,29.841696,18.361525,3,362,222,9,386,239,13.867751,2.479892,5.592080,1.679363,0.408088,1,0.817512,2.516640,0.0,57651.0,15280.698699,11289.105336
6,golgi,9,"(0.3891, 0.0799, 0.0799)",3.215435,20.061638,24.048551,6,245,295,11,258,309,7.994387,1.159269,6.896061,1.303352,0.513187,1,0.882798,2.023213,0.0,47854.0,15136.740899,10746.661481
7,golgi,12,"(0.3891, 0.0799, 0.0799)",3.479406,22.362202,10.476941,7,261,123,12,296,142,17.351123,2.983813,5.815083,1.786176,0.361504,1,0.821038,2.754456,0.0,65229.0,16481.509983,14048.393686
8,golgi,13,"(0.3891, 0.0799, 0.0799)",4.394240,21.686182,22.123955,8,252,241,15,285,312,44.216452,9.249325,4.780506,2.604377,0.227181,1,0.477876,6.876938,0.0,65535.0,16910.320988,10873.982074
9,golgi,14,"(0.3891, 0.0799, 0.0799)",4.077867,27.734440,14.141493,9,334,167,13,363,187,15.639289,3.249427,4.812937,1.837678,0.564224,1,0.895962,2.300854,0.0,40019.0,13202.179526,8539.269473


Morphology measurement (unscaled):


,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,golgi,1,"(1.0, 1.0, 1.0)",7.831360,317.720085,227.514599,1,274,181,16,371,280,10569.193359,13597.0,0.777318,29.612938,0.094394,1,0.274931,126.551070,0.0,65535.0,18106.068030,11022.022519
1,golgi,3,"(1.0, 1.0, 1.0)",9.153789,246.016838,226.117867,2,221,207,16,272,255,3806.012939,5345.0,0.712070,21.692837,0.155958,1,0.460220,53.886235,0.0,65535.0,24548.568756,14668.560908
2,golgi,4,"(1.0, 1.0, 1.0)",3.566398,322.132797,179.948692,2,305,171,6,342,192,995.510376,994.0,1.001519,12.382146,0.319820,1,0.822167,39.078061,0.0,39152.0,10127.904427,7046.602988
3,golgi,5,"(1.0, 1.0, 1.0)",3.272379,327.023018,238.337596,2,319,230,6,336,248,707.810669,782.0,0.905129,11.430609,0.638889,1,0.899885,19.426751,0.0,37253.0,14053.263427,8411.557879
4,golgi,6,"(1.0, 1.0, 1.0)",5.042625,282.848444,202.190798,3,266,190,9,300,216,1351.348999,1478.0,0.914309,14.132704,0.278658,1,0.773822,39.084957,0.0,50751.0,13597.763870,10113.348468
5,golgi,7,"(1.0, 1.0, 1.0)",5.009009,373.620621,229.887888,3,362,222,9,386,239,883.676392,999.0,0.884561,12.402873,0.408088,1,0.817512,26.521918,0.0,57651.0,15280.698699,11289.105336
6,golgi,9,"(1.0, 1.0, 1.0)",8.263383,251.173448,301.089936,6,245,295,11,258,309,435.202484,467.0,0.931911,9.625857,0.513187,1,0.882798,14.478316,0.0,47854.0,15136.740899,10746.661481
7,golgi,12,"(1.0, 1.0, 1.0)",8.941764,279.976705,131.172213,7,261,123,12,296,142,1112.407715,1202.0,0.925464,13.191736,0.361504,1,0.821038,34.207961,0.0,65229.0,16481.509983,14048.393686
8,golgi,13,"(1.0, 1.0, 1.0)",11.292807,271.512882,276.993827,8,252,241,15,285,312,3019.873047,3726.0,0.810487,19.234523,0.227181,1,0.477876,85.513274,0.0,65535.0,16910.320988,10873.982074
9,golgi,14,"(1.0, 1.0, 1.0)",10.479756,347.237586,177.052712,9,334,167,13,363,187,1093.926758,1309.0,0.835697,13.572099,0.564224,1,0.895962,28.765008,0.0,40019.0,13202.179526,8539.269473


-----
-----

## Define `get_morphology_metrics()` function

The following code includes an example of how the morphology measure steps above and combined into one function. This function can quantify the morphology of any objects, including organelles, interaction sites, and masks. It is applied in the batch process functions available in [_____]() notebooks.

This function can utilized from infer-subc using:
```python
infer_subc.utils.stats.get_morphology_metrics()
```

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block defines the `get_morphology_metrics()` function. It is applied below.

In [35]:
def _get_morphology_metrics(segmentation_img: np.ndarray, 
                           seg_name: str, 
                           intensity_img, 
                           mask: np.ndarray, 
                           scale: Union[tuple, None]=None):
    """
    Parameters
    ------------
    segmentation_img:
        an np.ndarray of segmented objects 
    seg_name: str
        a name or nickname (usually the segmentation file suffix) of the object being measured; this will be used for record keeping in the output table
    intensity_img:
        a single-channel np.ndarray contain gray scale values from the "raw" image the segmentation is based on; this image should be the same shape as the segmentation file
    mask:
        a binary np.ndarray mask of the area to measure from; this image should be the same shape as the segmentation file
    scale: tuple, optional
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)


    Regionprops measurements:
    ------------------------
    'label',
    'centroid',
    'bbox',
    'area',
    'equivalent_diameter',
    'extent',
    'euler_number',
    'solidity',
    'axis_major_length',
    'min_intensity',
    'max_intensity',
    'mean_intensity'

    Additional measurements:
    -----------------------
    'standard_deviation_intensity',
    'surface_area',
    'SA_to_volume_ratio`


    Returns
    -------------
    pandas dataframe of containing regionprops measurements (columns) for each object in the segmentation image (rows) and the regionprops object
    
    """
    ###################################################
    ## MASK THE ORGANELLE OBJECTS THAT WILL BE MEASURED
    ###################################################
    input_labels = _assert_uint16_labels(segmentation_img)
    input_labels = apply_mask(input_labels, mask)

    ##########################################
    ## CREATE LIST OF REGIONPROPS MEASUREMENTS
    ##########################################
    # start with LABEL
    properties = ["label", "centroid", "bbox", "area", 
                  "equivalent_diameter", "extent", "euler_number", "solidity", "axis_major_length",
                  "min_intensity", "max_intensity", "mean_intensity"]

    #######################
    ## ADD EXTRA PROPERTIES
    #######################
    def standard_deviation_intensity(region, intensities):
        return np.std(intensities[region])

    extra_properties = [standard_deviation_intensity]

    ##################
    ## RUN REGIONPROPS
    ##################
    props = regionprops_table(input_labels, 
                           intensity_image=intensity_img, 
                           properties=properties,
                           extra_properties=extra_properties,
                           spacing=scale)

    props_table = pd.DataFrame(props)

    ##################################################################
    ## RUN SURFACE AREA FUNCTION SEPARATELY AND APPEND THE PROPS_TABLE
    ##################################################################
    surface_area_tab = pd.DataFrame(_surface_area_from_props(input_labels, props, scale))

    #############################################
    ## RENAME AND ADD ADDITIONAL METADATA COLUMNS
    #############################################
    props_table.insert(0, "object", seg_name)
    props_table.rename(columns={"area": "volume"}, inplace=True)

    if scale is not None:
        round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
        props_table.insert(loc=2, column="scale", value=f"{round_scale}")
    else: 
        props_table.insert(loc=2, column="scale", value=f"{tuple(np.ones(segmentation_img.ndim))}") 

    props_table.insert(12, "surface_area", surface_area_tab)
    props_table.insert(14, "SA_to_volume_ratio", props_table["surface_area"].div(props_table["volume"]))


    return props_table

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block applies the function above to your test image. The settings specified above are applied here.

In [48]:
# run function above and compare it to the previous results
### SCALED ###
morph_table = _get_morphology_metrics(segmentation_img=seg, 
                                      seg_name=obj_seg_name, 
                                      intensity_img=int, 
                                      mask=mask, 
                                      scale=scale)

print("The quantification output here matches the output created above:")
print(f"Scaled: {morph_table.equals(test_props_table)}")

### UNSCALED ###
morph_table_unscaled = _get_morphology_metrics(segmentation_img=seg, 
                                      seg_name=obj_seg_name, 
                                      intensity_img=int, 
                                      mask=mask)

print(f"Unscaled: {morph_table_unscaled.equals(test_props_table_unscaled)}")

The quantification output here matches the output created above:
Scaled: True
Unscaled: True


-----
### **Congradulations! You've completed the `Morphology` method explanation notebook.**

Contrinue on to learn about the other methods included in `infer-subc`:
- [method_distribution](method_distribution.ipynb)
- [method_interactions](method_interactions.ipynb)

Alternatively, you can proceed to the batch processing notebooks to analyze your data:
- [batch_process_quantification](batch_process_quantification.ipynb)
